# EDA

подключаемся к базе данных

In [5]:
import logging
import os

import sqlite3
import pandas as pd

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(message)s",
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)
logger.setLevel('INFO')

db_file = '/srv/data/messages.db'

conn = sqlite3.connect(db_file, check_same_thread=True)

def run_sql(sql_str, db_con=conn, cols = None):
    with db_con as con:
        res = pd.DataFrame(con.execute(sql_str).fetchall(), columns=cols)
    return res
        
sql_str = """
    SELECT 
        name
    FROM 
        sqlite_master
    WHERE 
        name NOT LIKE 'sqlite_%';
"""
run_sql(sql_str, cols=['table_name'])

,table_name
0,tg_messages


Пример содержимого таблички с данными

In [6]:
TABLE_NAME = 'tg_messages'

table_df = pd.read_sql_query(f"SELECT * from {TABLE_NAME} LIMIT 10", conn)

table_df.head()

,id,msg,channel,msg_hash
0,2248,управляющая компания предлагает в аренду 1 одн...,rentinlimassol,314c320246f78c0db8ddb8489072f7ab
1,2246,управляющая компания предлагает в аренду целое...,rentinlimassol,6f3d312ef2c7f712ba0f761556492df0
2,2238,управляющая компания предлагает в аренду роско...,rentinlimassol,cb3cb9c0da0195b069392b849003d84f
3,2232,управляющая компания предлагает в аренду совре...,rentinlimassol,3f29b9e54210ea6c4ea3cafb27d04a9b
4,2224,управляющая компания предлагает в аренду роско...,rentinlimassol,2c697381b6caac91e7b684fa34b86025


Базовая статистика - сколько сообщений в табличке

In [7]:
sql_str = f"""
    SELECT
        COUNT(*) as num_messages,
        CAST(AVG(length(msg))  as integer) as avg_length,
        COUNT(DISTINCT channel) num_channels
    FROM {TABLE_NAME}
    LIMIT 10
"""

pd.read_sql_query(sql_str, conn)

,num_messages,avg_length,num_channels
0,27037,324,6


Далее в канале ищем "плохие" сообщения

In [12]:
from jinja2 import Template

def get_neg_samples_df():
    irrelevant_msg_ids = [
        153375, 130177, 152303, 156005, 152225, 152209, 152159, 152129,
        152831, 152766, 152740, 152697, 129161, 129139, 152628, 152556
    ]

    sql_str = Template(
        """
        SELECT 
            id, msg
        FROM {{ table }}
        WHERE id IN (
            {%- for msg_id in msg_ids -%} {{msg_id}} {{"," if not loop.last }} {% endfor %}
        )
        """
    ).render(msg_ids=irrelevant_msg_ids, table=TABLE_NAME)

    neg_samples_df = pd.read_sql_query(sql_str, conn)
    num_neg_samples = neg_samples_df.shape[0]

    logger.info('num rows: %d', num_neg_samples)
    
    return neg_samples_df

neg_samples_df = get_neg_samples_df()
neg_samples_df.head()

2022-12-12 17:40:57,523 num rows: 16


,id,msg
0,129161,продажа (собственник) лимассол 580 000 евро ...
1,129139,аренда город ларнака . район декелия . дом три...
2,152740,"продажа продается новая квартира за €310,000 ..."
3,152129,**mandarin park: новое высотное здание в лимас...
4,152159,**как получить визитерскую визу на кипре? инст...


Получилось 16 негативных примеров - маловато, попробуем быстро создать разметку данных

* векторизуем все примеры
* отранжируем неразмеченные примеры по схожести с предварительно размеченными примерами

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer


def get_train_set(limit = -1):
    negatives_df = get_neg_samples_df()
    if limit < 0:
        num_negatives = negatives_df.shape[0]
        num_positives = int( num_negatives / 0.1)
        limit = num_positives + num_negatives
    else:
        limit = 8*10**3
    irrelevant_msg_ids = negatives_df['id'].values.tolist()
    sql_str = Template(
        """
        SELECT 
            msg,
            length(msg) as len_msg,
            CASE
                WHEN id IN (
                        {%- for msg_id in msg_ids -%} {{msg_id}} {{"," if not loop.last }} {% endfor %}
                    )
                THEN 1
                ELSE 0
            END target
        FROM {{ table }}
        ORDER BY target DESC
        LIMIT {{ limit }}
        """
    ).render(msg_ids=irrelevant_msg_ids, table=TABLE_NAME, limit=limit)

    corpus_df = pd.read_sql_query(sql_str, conn)
    
    return corpus_df

class Pandas2CSR:
    def __init__(self):
        self.vectorizer = None
        self.txt_col = None
        self.anchor_elements = None
    
    def df_to_matrix(self, input_series):
        res = input_series.values.reshape(-1).tolist()
        
        return res
    
    def fit(self, input_df, text_column='msg'):
        csr_matrix_dataset = self.df_to_matrix(input_df[text_column])
        self.txt_col = text_column
        
        logger.info('num rows: %d', len(csr_matrix_dataset))

        self.vectorizer = TfidfVectorizer()
        X = self.vectorizer.fit_transform(csr_matrix_dataset)
        logger.info('sparse matrix %s', X.shape)
        
        return X
    
    def transform(self, input_df):
        corpus = self.df_to_matrix(input_df[self.txt_col])
        X = self.vectorizer.transform(corpus)
        
        logger.info('result matrix %s', X.shape)
        
        return X
    
    def generate_features(self, neg_samples_df):
        # сохраняем якорные элементы
        if self.anchor_elements is None:
            self.anchor_elements = self.transform(neg_samples_df)
        anchor_elems

corpus_df = get_train_set()
pandas2csr = Pandas2CSR()
raw_matrix = pandas2csr.fit(corpus_df)

corpus_df.head()

2022-12-12 17:41:17,263 num rows: 16
2022-12-12 17:41:17,288 num rows: 176
2022-12-12 17:41:17,318 sparse matrix (176, 2390)


,msg,len_msg,target
0,продажа (собственник) лимассол 580 000 евро ...,2009,1
1,аренда город ларнака . район декелия . дом три...,140,1
2,"продажа продается новая квартира за €310,000 ...",588,1
3,**mandarin park: новое высотное здание в лимас...,197,1
4,**как получить визитерскую визу на кипре? инст...,99,1


In [14]:
from sklearn.metrics.pairwise import euclidean_distances

neg_samples_csr = pandas2csr.transform(neg_samples_df)

distances = euclidean_distances(raw_matrix, neg_samples_csr)
logger.info(distances.shape)

2022-12-12 17:41:20,756 result matrix (16, 2390)
2022-12-12 17:41:20,762 (176, 16)


In [15]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression().fit(distances, corpus_df['target'])

Пробуем применить модель для быстрой разметки данных

In [16]:
corpus_df = get_train_set(limit=8*10**3)
raw_matrix = pandas2csr.transform(corpus_df)
distances = euclidean_distances(raw_matrix, neg_samples_csr)
neg_example_proba = lr.predict_proba(distances)

2022-12-12 17:41:24,826 num rows: 16
2022-12-12 17:41:25,249 result matrix (8000, 2390)


In [17]:
corpus_df['dummy_label'] = neg_example_proba[:,1]

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.expand_frame_repr', False)

scored_corpus_df = (
    corpus_df.query("len_msg > 0")
    [['msg', 'dummy_label']]
    .sort_values(by='dummy_label', ascending=False)
)

scored_corpus_df.head(270)

,msg,dummy_label
6486,ахахахахаха,0.987632
7005,halo,0.987632
7643,έχω μια επαγγελματική πρόταση και είναι επείγουσα,0.987632
4266,вот я,0.987632
7070,"вообще да, получали люди.",0.987632
...,...,...
269,"кипр, встречай **интернет-магазин epl diamond**! лучшие ювелирные изделия всемирного бренда теперь доступны к заказу он-лайн. покупайте ювелирные украшения на сайте [cy.epldiamond.com](http://cy.epldiamond.com/?utm_source=tgkipr_arenda) и получайте подарки! приятные цены и летние скидки вас порадуют! доставка по городу лимассол бесплатная, по кипру бесплатная при сумме заказа от 300 евро.",0.146483
4894,"******** ** [​](https://telegra.ph/file/0948a27872490e8a4a268.jpg)**#продажаземельногоучастка** **продаж** **зем**е**льного учаска** ———————— **#продажа**** **[**участка 12 000 ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)**[м²](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)** **[**limassol](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)******[**agios tychonas ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)** **стоимость 1**.**500 000 € ** **общая информация:** **обьект **№ **id: 0101 • mika** **•** **• **продаётся земельный участок с шикарным панорамным видом на море и город ,под инвестицию , в **агиос тихонас в **семи минутах от хайвей. **• **общая площадь [12 000 м²](https://telegra.ph/file/aece5c1a241defa34fa95.jpg) **• **площадь застройки [**5](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)** % **• **дорога подведена к участку,а также рядом проведена электросеть,вся инфраструктура рядом развита. эти участок может быть интересен тем кто желает ин...",0.146131
4625,"[​](https://telegra.ph/file/8388ab0aff36200470150.jpg)[​](https://telegra.ph/file/89c0136e2c0d406fe7156.jpg)**#аренда** ****** **аренд** **пентхауса ** ———————— **аренда 3-х спального современного пентхауса **[**limassol](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)******[**potamos germasoia ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)** **возле гостинцы аполлония стоимость: 5.500 €** **( два депозит одна аренда при оплате вперед стоимость будет снижена до 5**.**000 € )** **общая информация:** **обьект **№ **id: 0305** **•sa•** **• современный пентхаус в совершено новом красивом доме,на последнем этаже, в 150-200 метрах от моря. ** [**описание ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)****** **• общая крытая площадь **[**132 ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)**[м²](https://telegra.ph/file/aece5c1a241defa34fa95.jpg) **• kрытые веранды **[**39 ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)**[м²](https://telegra.p...",0.145923
53,**yoo limassol by philippe starck: эксклюзивный жилой комплекс в лимассоле** компания-застройщик презентовала свой новый проект. (фото) https://dom.com.cy/live/digest-56818/,0.145286


Видно, что в топе по нашему скору совсем нерелевантные сообщения (на первых позициях)

Ближе к концу списка сообщение `id=4625` уже релевантное

Сохраняем датасет для LabelStudio

In [166]:
scored_corpus_df.to_csv('/srv/data/scored_corpus.csv', index=False)
logger.info('%d lines saved', scored_corpus_df.shape[0])

INFO:__main__:6542 lines saved


# Тренировка модели

In [51]:
df = pd.read_csv('/srv/data/labeled_data_corpus.csv')

train_df = df[df['subset'] == 'train']
test_df = df[df['subset'] == 'test']
print(train_df.shape[0], train_df['label'].mean(), test_df.shape[0], test_df['label'].mean())

df.head()

5233 0.20045862793808522 1309 0.20091673032849502


,msg,label,subset
0,"здравствуйте. ишу 2х спальную квартиру в лимассоле. желательно гермасойя. семья из 2х взрослых и 2х детей. без животных. на длительный срок, бюджет до 1000-1500 евро. предложения в лс.",0,train
1,#сниму комнату в лимассоле или недалеко от него. с начала августа. любые предложения в лс,0,train
2,мошенник риэлторским услугам.,0,train
3,"**sales** reg.1053 lic.489/e **stylish apartment with sea view kissonerga. paphos** •total area: 85 m2 + balcony •bedrooms: 2 •bathrooms: 1 **€ 120,000** we have a lot to offer ================ **продажа** reg.1053 lic.489/e **стильные апартаменты вид на море •••kissonerga. пафос. ** •общая площадь: 85м2 + балкон •спальни: 2 •ванные комнаты: 1 **€ 120 000 ****+35726935826**** директ telegram 24/7** у нас есть что вам предложить",0,train
4,"важно: [valerii korol](tg://user?id=193474890), если ты не бот и не спамер, пройди проверку, нажав на кнопку, где есть",0,train


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import numpy as np
from sklearn.model_selection import ParameterGrid

X_train = train_df['msg'].values
y_train = train_df['label']

X_test = test_df['msg'].values
y_true = test_df['label']

grid = {
    'max_df': np.arange(0.01, 1.0, 0.01),
    'min_df': np.arange(1, 20, 1),
}

best_params = {'max_df': None, 'min_df': None}
best_score = 0.0

num_iters = len(ParameterGrid(grid))

cnt = 0
for param_values in ParameterGrid(grid):
    # fit 
    vectorizer = TfidfVectorizer(**param_values).fit(X_train)
    X_train_csr = vectorizer.transform(X_train)
    lr = LogisticRegression().fit(X_train_csr, y_train)
    # predict
    X_test_csr = vectorizer.transform(X_test)
    y_pred = lr.predict(X_test_csr)
    cur_score = f1_score(y_true, y_pred)
    if cur_score > best_score:
        best_score = cur_score
        best_params.update(param_values)
    if cnt % 250 == 0:
        logging.info(
            'iteration: %d of %d; %s; best_score= %.4f',
            cnt, num_iters, param_values, best_score
        )
    cnt = cnt + 1
print('best params: %s, best_score %.4f', best_params, best_score)

2022-12-12 21:01:06,151 iteration: 0 of 1881; {'max_df': 0.01, 'min_df': 1}; best_score= 0.6173
2022-12-12 21:03:26,644 iteration: 250 of 1881; {'max_df': 0.14, 'min_df': 4}; best_score= 0.8549
2022-12-12 21:05:50,131 iteration: 500 of 1881; {'max_df': 0.27, 'min_df': 7}; best_score= 0.8611
2022-12-12 21:08:15,245 iteration: 750 of 1881; {'max_df': 0.4, 'min_df': 10}; best_score= 0.8611
2022-12-12 21:10:37,462 iteration: 1000 of 1881; {'max_df': 0.53, 'min_df': 13}; best_score= 0.8611
2022-12-12 21:12:56,079 iteration: 1250 of 1881; {'max_df': 0.66, 'min_df': 16}; best_score= 0.8611
2022-12-12 21:15:11,239 iteration: 1500 of 1881; {'max_df': 0.79, 'min_df': 19}; best_score= 0.8611
2022-12-12 21:36:00,407 iteration: 1750 of 1881; {'max_df': 0.93, 'min_df': 3}; best_score= 0.8611


best params: %s, best_score %.4f {'max_df': 0.2, 'min_df': 18} 0.8610567514677103


In [68]:
df.reset_index().rename(columns={'index': 'msg_id'}).head()

,msg_id,msg,label,subset
0,0,"здравствуйте. ишу 2х спальную квартиру в лимассоле. желательно гермасойя. семья из 2х взрослых и 2х детей. без животных. на длительный срок, бюджет до 1000-1500 евро. предложения в лс.",0,train
1,1,#сниму комнату в лимассоле или недалеко от него. с начала августа. любые предложения в лс,0,train
2,2,мошенник риэлторским услугам.,0,train
3,3,"**sales** reg.1053 lic.489/e **stylish apartment with sea view kissonerga. paphos** •total area: 85 m2 + balcony •bedrooms: 2 •bathrooms: 1 **€ 120,000** we have a lot to offer ================ **продажа** reg.1053 lic.489/e **стильные апартаменты вид на море •••kissonerga. пафос. ** •общая площадь: 85м2 + балкон •спальни: 2 •ванные комнаты: 1 **€ 120 000 ****+35726935826**** директ telegram 24/7** у нас есть что вам предложить",0,train
4,4,"важно: [valerii korol](tg://user?id=193474890), если ты не бот и не спамер, пройди проверку, нажав на кнопку, где есть",0,train


In [69]:
df.reset_index().rename(columns={'index': 'msg_id'}).to_csv('/srv/data/labeled_data_corpus.csv', index=False)

In [ ]:
df = pd.read_csv('/srv/data/labeled_data_corpus.csv')[['msg', 'label']]

train_df = df[df['subset'] == 'train']
test_df = df[df['subset'] == 'test']
print(train_df.shape[0], train_df['label'].mean(), test_df.shape[0], test_df['label'].mean())